In [1]:
#tune the neural network to improve accuracy

In [2]:
!pip install scikeras
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.1 MB/s eta 0:00:00


In [3]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from lightgbm import LGBMRegressor, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from pandas import read_csv
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import keras_tuner
from tensorflow import keras
from keras_tuner import RandomSearch, GridSearch
import keras
from keras.layers.core.dense import utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.saving.legacy.saving_utils import model_call_inputs
from keras.layers import BatchNormalization, LayerNormalization
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from keras import Sequential
from keras.layers import Input, Dense
from keras.layers import Dropout, Activation


In [4]:
# read in dataset
df = pd.read_csv("https://github.com/microsoft/r-server-hospital-length-of-stay/raw/master/Data/LengthOfStay.csv")
df.head()

,eid,vdate,rcount,gender,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,...,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay
0,1,8/29/2012,0,F,0,0,0,0,0,0,...,192.476918,12.0,1.390722,30.432418,96,6.5,4,9/1/2012,B,3
1,2,5/26/2012,5+,F,0,0,0,0,0,0,...,94.078507,8.0,0.943164,28.460516,61,6.5,1,6/2/2012,A,7
2,3,9/22/2012,1,F,0,0,0,0,0,0,...,130.530524,12.0,1.065750,28.843812,64,6.5,2,9/25/2012,B,3
3,4,8/9/2012,0,F,0,0,0,0,0,0,...,163.377028,12.0,0.906862,27.959007,76,6.5,1,8/10/2012,A,1
4,5,12/20/2012,0,F,0,0,0,1,0,1,...,94.886654,11.5,1.242854,30.258927,67,5.6,2,12/24/2012,E,4


## 1. Data Cleaning

In [5]:
# type conversions
datetime_cols = ["vdate", "discharged"]
cat_cols = ["gender", "facid"]
bin_cols = ["dialysisrenalendstage",
            "asthma",
            "irondef",
            "pneum",
            "substancedependence",
            "psychologicaldisordermajor",
            "depress",
            "psychother",
            "fibrosisandother",
            "malnutrition",
            "hemo"]
num_cols = ["hematocrit",
            "neutrophils",
            "sodium",
            "glucose",
            "bloodureanitro",
            "creatinine",
            "bmi",
            "respiration"]

for date_col, cat_col in zip(datetime_cols, cat_cols):
    df[date_col] = pd.to_datetime(df[date_col], format="%m/%d/%Y")
    df[cat_col] = df[cat_col].astype("category")
df.head()

,eid,vdate,rcount,gender,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,...,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay
0,1,2012-08-29,0,F,0,0,0,0,0,0,...,192.476918,12.0,1.390722,30.432418,96,6.5,4,2012-09-01,B,3
1,2,2012-05-26,5+,F,0,0,0,0,0,0,...,94.078507,8.0,0.943164,28.460516,61,6.5,1,2012-06-02,A,7
2,3,2012-09-22,1,F,0,0,0,0,0,0,...,130.530524,12.0,1.065750,28.843812,64,6.5,2,2012-09-25,B,3
3,4,2012-08-09,0,F,0,0,0,0,0,0,...,163.377028,12.0,0.906862,27.959007,76,6.5,1,2012-08-10,A,1
4,5,2012-12-20,0,F,0,0,0,1,0,1,...,94.886654,11.5,1.242854,30.258927,67,5.6,2,2012-12-24,E,4


## 2. Feature Engineering

In [6]:
# number_of_issues transform function
def calculate_number_of_issues(df, bin_cols):
    df["numberofissues"] = df[bin_cols].sum(axis=1)
    return df

df = calculate_number_of_issues(df, bin_cols)

## 3. Modelling

In [7]:
df['rcount'].replace('5+',5, inplace=True)
df['gender'].replace('F',0, inplace=True)
df['gender'].replace('M',1, inplace=True)
df.drop(['facid'], axis=1)
df.drop(["discharged", "vdate", "eid"], axis=1, inplace=True)

In [8]:
labels = df["lengthofstay"]
features = df.loc[:,:'secondarydiagnosisnonicd9']

In [9]:
X = features.values.astype('int32')
Y = labels.values.astype('int32')

In [10]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

Trained Mean squared error 0.9300606306273562
Validate Mean squared error 0.92500595577059

In [11]:
input = 23
def build_model(hp):
    model = Sequential()
    model.add(Dense(input_dim=input, units=hp.Int('Dense1_neurons', min_value=1, max_value=500, step=1), activation='relu'))
    model.add(LayerNormalization())
    model.add(Dropout(rate=hp.Float('D1_rate', min_value=0.1, max_value=0.9, step=0.1)))
    model.add(Dense(units=hp.Int('Dense2_neurons', min_value=1, max_value=500, step=1), activation='relu'))
    model.add(LayerNormalization())
    model.add(Dropout(rate=hp.Float('D2_rate', min_value=0.1, max_value=0.9, step=0.1)))
    model.add(Dense(units=hp.Int('Dense3_neurons', min_value=1, max_value=500, step=1), activation='relu'))
    model.add(LayerNormalization())
    model.add(Dropout(rate=hp.Float('D3_rate', min_value=0.1, max_value=0.9, step=0.1)))
    model.add(Dense(units=hp.Int('Dense4_neurons', min_value=1, max_value=500, step=1), activation='relu'))
    model.add(LayerNormalization())
    model.add(Dense(1, activation='relu'))

    # Compile model
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    return model

# Import the necessary libraries to use Keras Tuner
from kerastuner.tuners import RandomSearch

# Define the hyperparameter tuner
tuner = RandomSearch(
    build_model,  # Pass the function that defines the model
    objective='val_accuracy',  # Specify the metric to optimize
    max_trials=20,  # Number of hyperparameter combinations to try
    directory='/content/drive/MyDrive/Colab Notebooks/model tuning',  # Directory to store the results
    project_name='my_model_tuning'  # Name of the tuning project
)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

tuner.search(X_train, Y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

Trial 20 Complete [00h 00m 43s]
val_accuracy: 0.17733334004878998

Best val_accuracy So Far: 0.17733334004878998
Total elapsed time: 00h 19m 16s


In [12]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("Dense 1 nerons", best_hps.get('Dense1_neurons'))
print("Dense 2 nerons", best_hps.get('Dense2_neurons'))
#print("Dense 3 nerons", best_hps.get('Dense3_neurons'))
#print("Dense 4 nerons", best_hps.get('Dense4_neurons'))
#print("Dense 5 nerons", best_hps.get('Dense5_neurons'))
print("D1 rate", best_hps.get('D1_rate'))
#print("D2 rate", best_hps.get('D2_rate'))
#print("D3 rate", best_hps.get('D3_rate'))
#print("D4 rate", best_hps.get('D4_rate'))


Dense 1 nerons 143
Dense 2 nerons 70
D1 rate 0.1


In [13]:
import keras_tuner as kt


tuner = kt.GridSearch(model_b,
                     objective='val_accuracy',
                     max_trials=10,
                     directory='/content/drive/MyDrive/Colab Notebooks/model tuning',
                     project_name='RNN modele')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, Y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])


NameError: ignored

In [ ]:
model_a = Sequential()
model_a.add(Dense(500, input_dim=23, activation= "relu"))
model_a.add(Dense(100, activation= "relu"))
model_a.add(Dense(50, activation= "relu"))
#model_a.add(Dense(25, activation= "relu"))
model_a.add(Dense(1))
model_a.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_a.fit(X_train, Y_train, epochs=20)
kfold = KFold(n_splits=10)
pred_train= model_a.predict(X_train)
print('Trained Mean squared error',np.sqrt(mean_squared_error(Y_train,pred_train)))

pred= model_a.predict(X_test)
print('Validate Mean squared error',np.sqrt(mean_squared_error(Y_test,pred)))


In [ ]:
#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_sv = sc_X.fit_transform(X_train)
y_sv = sc_y.fit_transform(Y_train)
y_sv.reshape(-1,1)
#4 Fitting the Support Vector Regression Model to the dataset
# Create your support vector regressor here
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
regressor = SVR(kernel='linear')
regressor.fit(X_sv,y_sv)
